## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [ ]:
import pandas as pd

import numpy as np

In [1]:
data = pd.read_csv('/datasets/data.csv')

data.to_csv('data.csv')

print(data.info())

display(data.head(10))

display(data.groupby('income_type').agg({'children':['min', 'max', 'median'], 'days_employed':['max', 'min', 'median'],'dob_years':['max', 'min', 'median'],'total_income':['max', 'min', 'median'], }))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


children             days_employed                 \
                     min max median            max            min   
income_type                                                         
безработный            0   1    0.5  395302.838654  337524.466835   
в декрете              2   2    2.0   -3296.759962   -3296.759962   
госслужащий           -1  20    0.0     -39.954170  -15193.032201   
компаньон             -1  20    0.0     -30.195337  -17615.563266   
пенсионер             -1  20    0.0  401755.400475  328728.720605   
предприниматель        0   0    0.0    -520.848083    -520.848083   
сотрудник             -1  20    0.0     -24.141633  -18388.949901   
студент                0   0    0.0    -578.751554    -578.751554   

                               dob_years             total_income  \
                        median       max min median           max   
income_type                                                         
безработный      366413.652744        45  31   38.0  2.027225e+05   
в декрете         -3296.759962        39  39   39.0  5.382913e+04   
госслужащий       -2689.368353        75   0   40.0  9.104515e+05   
компаньон         -1547.382223        74   0   39.0  2.265604e+06   
пенсионер        365213.306266        74   0   60.0  7.351033e+05   
предприниматель    -520.848083        58  27   42.5  4.991631e+05   
сотрудник         -1574.202821        74   0   39.0  1.726276e+06   
студент            -578.751554        22  22   22.0  9.820163e+04   

                                               
                           min         median  
income_type                                    
безработный       59956.991984  131339.751676  
в декрете         53829.130729   53829.130729  
госслужащий       29200.077193  150447.935283  
компаньон         28702.812889  172357.950966  
пенсионер         20667.263793  118514.486412  
предприниматель  499163.144947  499163.144947  
сотрудник         21367.648356  142594.396847  
студент           98201.625314   98201.625314

In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В столбцах days_employed и income_type имеются пропуски данных, более 2000 строк, что составляет около 10% от общего числа, следовательно требуется замена пропущенных значений, удаление такоего большого количества значений может оказать серьезное влияние на конечные результаты. В столбце education записи выполнены как в нижнем, так и в верхнем регистре, требуется привести все строки к единообразию. Столбец days_emlpoyed содержит либо отрицательные значения стажа, либо неправдоподобные положительные значения для пенсионеров и безработных. Отрицательные значения стажа могли возникнуть, допустим, если при расчете стажа из даты устройства на работу отнималась дата увольнения. Дополнительно стоит отметить, что в слолбце dob_years имеются нулевые значения возраста, от них тоже необходимо будет избавиться. В столбце children имеются неправдоподобные количества детей в размере "-1" или "20".

### Шаг 2. Предобработка данных

## Обработка пропусков

In [3]:
display(data.loc[data['total_income'].isna()].head(10)) # просматриваем срез по пропущенным значениям дохода

#смотрим метрики распределения пропущенных значений дохода по категориям к распределению по категориям датафрейма в целом
print(data.loc[data['total_income'].isna(), 'income_type'].value_counts(normalize = True)/data['income_type'].value_counts(normalize = True))
print(data.loc[data['total_income'].isna(), 'family_status'].value_counts(normalize = True)/data['family_status'].value_counts(normalize = True))
print(data.loc[data['total_income'].isna(), 'gender'].value_counts(normalize = True)/data['gender'].value_counts(normalize = True))

data.loc[data['dob_years'] == 0, 'dob_years'] = round(data['dob_years'].mean()).astype('int64') #заменяем пропущенные значения возраста средним

income_median = data.groupby('income_type')['total_income'].median() # определяем медиану дохода для каждого типа занятости

income_type_dict = data['income_type'].unique() # словарь типов занятости

days_employed_median = data.groupby('dob_years')['days_employed'].median() #медиана трудового стажа по возрасту

dob_years_dict = data['dob_years'].unique() #словарь возрастов

def nan_to_median(row): #опеределяем функцию замены пропущенных значений в части дохода и трудового стажа
    if pd.isna(row['days_employed']):
        for i in dob_years_dict:
            if i == row['dob_years']:
                row['days_employed'] = days_employed_median[i]
    if pd.isna(row['total_income']):
        for i in income_type_dict:
            if i == row['income_type']:
                row['total_income'] = income_median[i]
    return row
    
data = data.apply(nan_to_median, axis=1)

data['days_employed'] = data['days_employed'].abs() #приводим столбец стажа к положительным значения

#обрабатываем некорректные значения стоблца children
def children_correction(row):
    if (row['children'] == -1) | (row['children'] == 20):
        row['children'] = 0
    return row

data['children'] = data.apply(children_correction, axis = 1)

print(data.info()) #проверяем заполнение пропущенных значений в стобцах с доходом и трудовым стажем

print(data['children'].value_counts()) #выводим список уникальных значений столбца children

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


безработный             NaN
в декрете               NaN
госслужащий        0.997575
компаньон          0.989137
пенсионер          1.060466
предприниматель    4.950552
сотрудник          0.983966
студент                 NaN
Name: income_type, dtype: float64
женат / замужем          0.989311
гражданский брак         1.047711
Не женат / не замужем    1.013693
в разводе                0.927970
вдовец / вдова           0.979797
Name: family_status, dtype: float64
F      1.032118
M      0.937399
XNA         NaN
Name: gender, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null object
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type     

### Вывод

Согласно рассчитанным метрикам по отношению распределения пропущенных значений к распределению по датафрейму, ни одна из групп по трудоустройству, семейному положению и полу не выделяется из общего списка, за исключением крайне малочисленных групп (например, предприниматели). Стоит отметить, что у всех клиентов с пропусками значений дохода также присутствуют пропуски в трудовом стаже. Такая ситуация могла возникнуть, если клиент не сообщил банку данные о своем месте трудоустройства. Как вариант, все остальные клиенты в датафрейме, доход которых нам известен, получают доход на карту банка, а те, кто имеют пропущенные значения дохода, не являются зарплатными клиентами.

Все записи с нулевым значением возраста были замены на средний возраст, пропуски в столбцах с доходом и трудовым стажем были заменены медианным значением при группировке по типу занятости и возраста соответственно. Пропущенные значения стажа были заменены медианным значением стажа для клиентов соответствующего возраста. Так же значения стажа для удобства были переведены в положительные. Пропущенные значения дохода клиентов были заменены медианным значением для клиентов соответствующего типа занятости.
Некорректные значения количества детей были заменены на 0, т.е на отсутсвие детей у клиента, т.к эта группа клиентов является самой крупной и наиболее вероятной в наборе данных.

### Замена типа данных

In [4]:
#выполняем преобразование вещественных значений в целочисленные

data['days_employed'] = data['days_employed'].astype('int64')

data['total_income'] = data['total_income'].astype('int64')

data.info() #проверяем результат преобразований

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null object
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(6)
memory usage: 2.0+ MB


### Вывод

После изучения info() на шаге №1, было обнаружено, что столбцы с доходом и образованием представлены значениями в формате вещественных чисел. Исключительно для удобства дальнейшего изучения этих данных можно привести значения к целочисленным. Так как значения были вещественными для их преобразования к целым был использован метод astype(). 

### Обработка дубликатов

In [5]:
data['education'] = data['education'].str.lower() #приводим столбец образование к нижнему регистру

print('Количество найденных дубликатов:',data.duplicated().sum()) #находим суммарное количество дубликатов в таблице и выводим на экран

data = data.drop_duplicates().reset_index(drop = True) #удаляем дубликаты и обновляем индексы строк датафрейма

Количество найденных дубликатов: 71


### Вывод

Перед поиском дубликатов значения столбца образования были приведены к единому регистру, после этого, посредством последовательного применения методов duplicated() и sum() к исходному датафрейму, были найдена 71 дублирующаяся строка. 
Удаление дубликатов было выполнено применением метода drop_duplicates() к исходному датафрейму с последующим обновлением индексов строк. 

### Лемматизация

In [6]:
from pymystem3 import Mystem

str_for_lemmatize = ' '.join(data['purpose'].unique()) #собираем все уникальные значения цели кредита в одину строку

lemmas = Mystem().lemmatize(str_for_lemmatize) #производим лемматизацию

from collections import Counter

print(Counter(lemmas)) #выводим список уникальных слов, полученных после лемматизации







Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


### Вывод

Для лемматизации целей кредита был получен список из всех ункальных значений столбца purpose, после все значения этого столбца были объеденины в одну строку, по которой и проводилась лемматизация.

Из полученного списка лемматизированных слов можно сделать вывод, что основными целями для получения кредита являются: недвижимость, автомобиль, образование, свадьба.

### Категоризация данных

In [7]:
def purpose_grouping(row): #определим функцию для группировки по целям кредита
    if ('недвиж' in row['purpose']) | ('жил' in row['purpose']) | ('строит' in row['purpose']) :
        return 'недвижимость'
    elif ('образ' in row['purpose']):
        return 'образование'
    elif ('авто' in row['purpose']):
        return 'авто'
    elif ('свад' in row['purpose']):
        return 'свадьба'
    else:
        return 'другое'
    
data['purpose_group'] = data.apply(purpose_grouping, axis = 1)

print(data['purpose_group'].value_counts()) #выводим число сгруппированных записей по каждой категории для целей кредита

meadian_income = data['total_income'].median() #рассчитываем медианный доход по всему датафрейму

def income_grouping(row): #определим фукцию группировки по уровню дохода 
    if (row['total_income'] > (2*meadian_income)): 
        return 'высокий'
    elif (row['total_income'] < (0.75*meadian_income)):
        return 'низкий'
    else:
        return 'средний'
data['income_level'] = data.apply(income_grouping, axis = 1)

print(data['income_level'].value_counts()) #выводим число сгруппированных записей по каждой категории для дохода

недвижимость    10811
авто             4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64
средний    14401
низкий      5296
высокий     1757
Name: income_level, dtype: int64


### Вывод

Основанием для категоризации данных послужили вопросы, на которые в дальнейшем необходимо ответить в ходе выполнение проекта. Разумно было сгруппировать множества целей получение кредита и определить группы по доходу клиентов. При группировке по целям кредита были использованы основные категории полученные после лемматизации. По уровню дохода клиенты были распределены на 3 группы ( с высоким доходом, со средним доходом, с низким доходом). Организация экономического сотрудничества и развития определяет средний доход, как доход, укладывающийся в 75-200% от медианного дохода, по такому же принципу были сформированы группы по доходу.
В итоге получилось, что чаще всего клиенты берут кредит на недвижимость, а реже всего - на свадьбу. Подовляющее число клиентов банка как раз таки могут отнеси себя людям со средним доходом. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
#children_all = data['children'].value_counts()

#children_debt = data[data['debt']== 1].groupby('children')['debt'].count()

#children_debt_percent = round(children_debt/children_all*100, 2)

#children_result = pd.concat([children_all, children_debt, children_debt_percent], axis = 1, sort = True).dropna()

#children_result = children_result.set_axis(['all','debt_quantity','debt_percent'], axis = 'columns', inplace = False)

#children_result['debt_quantity'] = children_result['debt_quantity'].astype('int64')

#display(children_result.sort_values('debt_percent', ascending = False))

data.pivot_table(index = 'children', values = 'debt' , aggfunc = ['count','sum', lambda s: round(s.sum()/s.count()*100, 2)] , dropna = True)

,count,sum,<lambda>
,debt,debt,debt
children,,,
0,14214,1072,7.54
1,4808,444,9.23
2,2052,194,9.45
3,330,27,8.18
4,41,4,9.76
5,9,0,0.00


### Вывод

В приведенной выше выведены сгруппированные по числу детей значения: в столбце "count debt" - все количество клиентов; в стоблце "sum debt" - количество клиентов, имеющих задолженность по возврату кредита; "lambda debt" - процент клиентов от общего числа группы, имеющих задолженнность по возврату кредита. 

Можно сделать вывод о том, что бездетные клиенты имеют лучшие показатели по возврату кредитов. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
#family_all = data['family_status'].value_counts() 

#family_debt = data[data['debt'] == 1].groupby('family_status')['debt'].count()

#family_debt_percent = round(family_debt/family_all*100, 2)

#family_result = pd.concat([family_all, family_debt, family_debt_percent], axis = 1, sort = True)

#family_result = family_result.set_axis(['all','debt_quantity','debt_percent'], axis = 'columns', inplace = False)

#print(family_result.sort_values('debt_percent', ascending = False))

data.pivot_table(index = 'family_status', values = 'debt' , aggfunc = ['count','sum', lambda s: round(s.sum()/s.count()*100, 2)] , dropna = True)


,count,sum,<lambda>
,debt,debt,debt
family_status,,,
Не женат / не замужем,2810,274,9.75
в разводе,1195,85,7.11
вдовец / вдова,959,63,6.57
гражданский брак,4151,388,9.35
женат / замужем,12339,931,7.55


### Вывод

В приведенной выше таблице выведены сгруппированные по семейному положению значения: в столбце "count debt" - все количество клиентов; в стоблце "sum debt" - количество клиентов, имеющих задолженность по возврату кредита; "lambda debt" - процент клиентов от общего числа группы, имеющих задолженнность по возврату кредита. 

Из таблицы видно, что хуже всего возвращают кредиты холостые и незамужние клиенты, и живущие в гражданском браке. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
#income_all = data['income_level'].value_counts()

#income_debt = data[data['debt']==1].groupby('income_level')['debt'].count()

#income_debt_percent = round(income_debt/income_all*100, 2)

#income_result = pd.concat([income_all, income_debt, income_debt_percent], axis = 1, sort = True)

#income_result = income_result.set_axis(['all','debt_quantity','debt_percent'], axis = 'columns', inplace = False)

#print(income_result.sort_values('debt_percent', ascending = False))

data.pivot_table(index = 'income_level', values = 'debt' , aggfunc = ['count','sum', lambda s: round(s.sum()/s.count()*100, 2)] , dropna = True)

,count,sum,<lambda>
,debt,debt,debt
income_level,,,
высокий,1757,124,7.06
низкий,5296,421,7.95
средний,14401,1196,8.30


### Вывод

В приведенной выше таблице выведены сгруппированные по семейному положению значения: в столбце "count debt" - все количество клиентов; в стоблце "sum debt" - количество клиентов, имеющих задолженность по возврату кредита; "lambda debt" - процент клиентов от общего числа группы, имеющих задолженнность по возврату кредита.

Худшие показатели по возврату задолженности имеют клиенты со средним доходом, думаю это связано с тем, что таким клиентами банки охотнее одобряют кредиты, чем клиентам с низким доходом. Ожидаемо, клиенты с высоким достатком имеют меньше всего проблем с возвратом кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
#purpose_all = data['purpose_group'].value_counts()

#purpose_debt = data[data['debt'] == 1].groupby('purpose_group')['debt'].count()

#purpose_debt_percent = round(purpose_debt/purpose_all*100, 2)

#purpose_result = pd.concat([purpose_all, purpose_debt, purpose_debt_percent], axis = 1, sort = True)

#purpose_result = purpose_result.set_axis(['all','debt_quantity','debt_pecrent'], axis = 'columns', inplace = False)

#print(purpose_result.sort_values('debt_pecrent', ascending = False))

data.pivot_table(index = 'purpose_group', values = 'debt' , aggfunc = ['count','sum', lambda s: round(s.sum()/s.count()*100, 2)] , dropna = True)

,count,sum,<lambda>
,debt,debt,debt
purpose_group,,,
авто,4306,403,9.36
недвижимость,10811,782,7.23
образование,4013,370,9.22
свадьба,2324,186,8.00


### Вывод

В приведенной выше таблице выведены сгруппированные по семейному положению значения: в столбце "count debt" - все количество клиентов; в стоблце "sum debt" - количество клиентов, имеющих задолженность по возврату кредита; "lambda debt" - процент клиентов от общего числа группы, имеющих задолженнность по возврату кредита.

Нетрудно заметить, чаще всего не возвращаются кредиты, целью которых являются операции с автомобилями. Клиенты же берущие займ на операции с недвижимостью ответсвеннее всех относятся к своим долговым обязательствам.

### Шаг 4. Общий вывод

На первом шаге исследования исходные данные были изучены, найдены аномальные значения и пропуски в данных. На втором шаге выполнения исследования исходные данные были обработаны: заменены все пропуски и некорректные значения, способные повлиять на конечный результат исследования, удалены дублирующиеся строки. С помощью лемматизации определены основные категории для целям получения кредита, а так же группы по уровню дохода клиентов. На третьем шаге рассчитаны метрики, которые описывают зависимости возврата кредита в срок от количества детей клиента, его семейного положения, дохода клиента, цели получения кредита.

По результатам исследования можно рекомендовать банку с осторожностью одобрять кредиты клиентам с детьми, неженатым и незамужним, клиентам со средним уровнем дохода, а так же тем, кто берет кредит для приобретения автомобиля или получения образования. Более благонадежными клиентами будут являться клиенты с высоким уровнем дохода, бездетные, состоящие или состаявшие в браке, берущие займ на операции с недвижимостью.  